In [1]:
import numpy as np
import cv2
from scipy.fftpack import dct, idct
from scipy.special import gamma

def embed_watermark_dct(original_img, watermark_bits, alpha_D=0.025, T_D=10, seed=42):
    # Convert to grayscale if necessary
    if len(original_img.shape) == 3:
        original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    h, w = original_img.shape
    h = h // 8 * 8
    w = w // 8 * 8
    original_img = original_img[:h, :w]
    watermarked_img = np.zeros_like(original_img, dtype=np.float32)
    
    # Generate zigzag order for mid-frequency coefficients
    zigzag = [(0,0), (0,1), (1,0), (2,0), (1,1), (0,2), (0,3), (1,2),
              (2,1), (3,0), (4,0), (3,1), (2,2), (1,3), (0,4), (0,5)]
    selected_coeffs = zigzag[5:13]  # Mid-frequency indices
    
    # Split into 8x8 blocks and process
    np.random.seed(seed)
    bit_idx = 0
    for i in range(0, h, 8):
        for j in range(0, w, 8):
            block = original_img[i:i+8, j:j+8].astype(np.float32)
            dct_block = dct(dct(block.T, norm='ortho').T, norm='ortho')
            
            # Embed each bit in selected coefficients
            for (x, y_coeff) in selected_coeffs:
                if bit_idx >= len(watermark_bits):
                    break
                coeff = dct_block[x, y_coeff]
                f = alpha_D * max(T_D, abs(coeff))
                if watermark_bits[bit_idx] == 1:
                    dct_block[x, y_coeff] += f
                else:
                    dct_block[x, y_coeff] -= f
                bit_idx += 1
            
            # Inverse DCT
            idct_block = idct(idct(dct_block.T, norm='ortho').T, norm='ortho')
            watermarked_img[i:i+8, j:j+8] = idct_block
    
    watermarked_img = np.clip(watermarked_img, 0, 255).astype(np.uint8)
    return watermarked_img

In [2]:
def detect_watermark_dct(watermarked_img, num_bits, alpha_D=0.025, T_D=10, seed=42):
    if len(watermarked_img.shape) == 3:
        watermarked_img = cv2.cvtColor(watermarked_img, cv2.COLOR_BGR2GRAY)
    h, w = watermarked_img.shape
    h = h // 8 * 8
    w = w // 8 * 8
    watermarked_img = watermarked_img[:h, :w]
    
    zigzag = [(0,0), (0,1), (1,0), (2,0), (1,1), (0,2), (0,3), (1,2),
              (2,1), (3,0), (4,0), (3,1), (2,2), (1,3), (0,4), (0,5)]
    selected_coeffs = zigzag[5:13]
    
    # Collect DCT coefficients
    blocks = []
    for i in range(0, h, 8):
        for j in range(0, w, 8):
            block = watermarked_img[i:i+8, j:j+8].astype(np.float32)
            dct_block = dct(dct(block.T, norm='ortho').T, norm='ortho')
            blocks.append(dct_block)
    blocks = np.array(blocks)
    
    # Estimate GGD parameters (sigma and c)
    all_coeffs = [block[coeff] for block in blocks for coeff in selected_coeffs]
    sigma = np.std(all_coeffs)
    c = 0.7  # Approximation for shape parameter
    
    # Precompute GGD constants
    beta = (1 / sigma) * np.sqrt(gamma(3/c) / gamma(1/c))
    A = (beta * c) / (2 * gamma(1/c))
    
    detected_bits = []
    np.random.seed(seed)
    coeff_indices = np.random.permutation(len(selected_coeffs) * len(blocks))
    
    for bit in range(num_bits):
        likelihood = 0.0
        for idx in coeff_indices[bit::num_bits]:
            block_idx = idx // len(selected_coeffs)
            coeff_idx = idx % len(selected_coeffs)
            x, y = selected_coeffs[coeff_idx]
            y_val = blocks[block_idx][x, y]
            
            f_est = alpha_D * max(T_D, abs(y_val))
            x_hat1 = y_val - f_est
            x_hat0 = y_val + f_est
            
            # Log probabilities under GGD
            ln_p1 = np.log(A) - (beta * np.abs(x_hat1)) ** c
            ln_p0 = np.log(A) - (beta * np.abs(x_hat0)) ** c
            
            # Jacobian terms (approximated)
            df_dx1 = alpha_D if abs(x_hat1) >= T_D else 0.0
            J1 = 1.0 / (1 + df_dx1)
            df_dx0 = alpha_D if abs(x_hat0) >= T_D else 0.0
            J0 = 1.0 / (1 - df_dx0)
            
            likelihood += (ln_p1 - ln_p0) + (np.log(J1) - np.log(J0))
        
        detected_bits.append(1 if likelihood > 0 else 0)
    
    return detected_bits

In [3]:
# Generate a random watermark (e.g., 256 bits)
watermark_bits = np.random.randint(0, 2, 256)

# Embed watermark
original_img = cv2.imread('lenna.png')
watermarked_img = embed_watermark_dct(original_img, watermark_bits)

# Detect watermark
detected_bits = detect_watermark_dct(watermarked_img, len(watermark_bits))

# Calculate Bit Error Rate
ber = np.mean(np.abs(watermark_bits - detected_bits))
print(f"Bit Error Rate: {ber:.4f}")

Bit Error Rate: 0.4805
